In [63]:
import pandas as pd
import json
import numpy as np
import collections

## TODO. 
0) Обдумать логику чекеров.  
1) Добавить проверку файла на существование  
2) Добавить добавление нового правила.  
-) Траснпанировать словарь conformity  
4) Описать правила.   
5) Сепаратор  
6) Поднимать ошибку когда есть неоднозначность  
7) Если сможем сами понимать значение столбцов и соотносить их сами то будет круто  

In [65]:
indefecators_file = "Identifiers.json"
with open(indefecators_file, "r") as read_file:
    indefecators = json.load(read_file)

In [67]:
indefecators = indefecators["Identifiers"]
indefecators

{'Capacity': ['pF',
  'pf',
  'пФ',
  'пф',
  'nF',
  'nf',
  'нФ',
  'нф',
  'uF',
  'uf',
  'мкФ',
  'мФ',
  'мкф',
  'мф'],
 'Resistance': ['k',
  'K',
  'к',
  'К',
  'M',
  'm',
  'м',
  'М',
  'Ohm',
  'ohm',
  'Ом',
  'ом'],
 'Inductance': ['H', 'h', 'Гн', 'гн', 'uH', 'uh', 'мкГн', 'мкгн']}

In [2]:
void_checker = lambda x: None
identical_checker = lambda x: x 

class Rules():
    checker = {
        "Component": [void_checker],
        "Capacity": [void_checker],
        "Resistance": [void_checker],
        "Inductance": [void_checker],
        "Tolerance": [void_checker],
        "Body": [void_checker],
        "Value": [void_checker],
        "Location": [void_checker],
    }
        
    
    def check_if(self, goal, values_to_check):
        if goal not in ["Capacity", "Resistance", "Inductance"] and len(values_to_check) == 1:
            return identical_checker(values_to_check[0])
        else:
            answ = []
            for checker in self.checker[goal]:
                answ.append(checker(values_to_check))
            return answ
            #return self.checker[goal](values_to_check)

In [3]:
class Specification():
    def __init__(self, path, rows_to_skip = 0, separator = "/" , columns_to_work_with = None):
        # add check if file exists
        
        ###
        if columns_to_work_with is not None:
            self.data = pd.read_excel(path)[columns_to_work_with][rows_to_skip:]
        else:
            self.data = pd.read_excel(path)[rows_to_skip:]
        ###
        
        self.parsed_data = pd.DataFrame(
            columns=["Component_type",
                     "Name",
                     "Capacity",
                     "Resistance",
                     "Inductance",
                     "Tolerance",
                     "Body",
                     "Value",
                     "Location",
                    ])
        
        all_col = list(self.data.columns)
        
        self.conformity = {
            "Component_type": all_col,
            "Name": all_col,
            "Capacity": all_col,
            "Resistance": all_col,
            "Inductance": all_col,
            "Tolerance": all_col,
            "Body": all_col,
            "Value": all_col,
            "Location": all_col,
        }
        
        self.rules = Rules()
        
    def make_conformity(self, input_col: str, output_col: str):
        self.conformity[output_col] = [input_col]
        
        
    def call_checker(self, goal, data):
        # поднимать ошибку при неоднозначном результате
        columns = self.conformity[goal]
        data_to_checker = []
        for col in columns:
            data_to_checker.append(data[col])
        if len(data_to_checker) == 1:
            return self.rules.check_if(goal, data_to_checker)
        else:
            possible_answ = []
            for d in data_to_checker:
                possible_answ.append(self.rules.check_if(goal, [d]))
            #possible_answ = np.array([possible_answ])
            #return max(set(possible_answ), key = possible_answ.count)
            return possible_answ[0]
        #return self.checker[goal](data_to_checker)
        
    def parse(self):
        for i in range(len(self.data)):
            current_row = self.data.iloc[i]
            self.parsed_data = self.parsed_data.append(
                {
                "Component_type": self.call_checker("Component_type", current_row),
                "Name": self.call_checker("Name", current_row),
                "Capacity": self.call_checker("Capacity", current_row),
                "Resistance": self.call_checker("Resistance", current_row),
                "Inductance": self.call_checker("Inductance", current_row),
                "Tolerance": self.call_checker("Tolerance", current_row),
                "Body": self.call_checker("Body", current_row),
                "Value": self.call_checker("Value", current_row),
                "Location": self.call_checker("Location", current_row),
                },
                ignore_index=True
            )
    
    def change_rule(self, goal, rule):
        self.rules.checker[goal] = [rule]
    
    def add_rule(self, goal, rule):
        # rule is the function
        self.rules.checker[goal].append(rule)
        
    def return_parsed(self):
        if len(self.parsed_data()) == 0:
            self.parse()
        return self.parsed_data()
    

In [11]:
testParser = Specification("data/Спецификации SNRM-7140.xlsx", rows_to_skip=2)
testParser2 = Specification("data/Спецификации SNRM-7140.xlsx", rows_to_skip=2)

In [36]:
testParser.data\

№ Qty                                  Центральная плата  \
2      1.0   1                                                 C1   
3      2.0   2                                           C81 C124   
4      3.0  12             C6-7 C13 C20 C51-52 C83 C89\n C176-179   
5      NaN   2                                             C66-67   
6      4.0  12  C84 C90 C92 C102 C116 C118 \nC127 C129 C162-16...   
..     ...  ..                                                ...   
134  118.0   1                                                 D1   
135  119.0   1                                                 D2   
136  120.0   1                                                 U3   
137  121.0   1                                                 U2   
138  122.0   1                                                 J1   

           Value Tolerance Part Number                 Description  \
2    0.01uf/630v       0.2         NaN    SMD Capacitor 1206(3216)   
3     0.1uf/100v       0.2         NaN    SMD Capacitor 1206(3216)   
4      0.1uf/16v       0.1         NaN    SMD Capacitor 0603(1608)   
5      0.1uf/16v       0.2         NaN    SMD Capacitor 0603(1608)   
6      0.1uf/16v       0.2         NaN    SMD Capacitor 0805(2012)   
..           ...       ...         ...                         ...   
134          NaN       NaN   1PS76SB10                       Diode   
135          NaN       NaN  BZX84-C4V3                 Zener Diode   
136          NaN       NaN      LM224D  Quad Operational Amplifier   
137          NaN       NaN   PAM8406DR   5W STEREO AUDIO AMPLIFIER   
138          NaN       NaN  XH-D-03-LF     Discrete Wire Connector   

         Manufacturer              Remark  
2                 NaN                 NaN  
3                 NaN                 NaN  
4                 NaN                 NaN  
5                 NaN                 NaN  
6                 NaN                 NaN  
..                ...                 ...  
134               NXP                 NaN  
135               NXP                 NaN  
136  ON Semiconductor                 NaN  
137       Diodes Inc.                 NaN  
138       Morethanall  DS1069-3MV Connfly  

[137 rows x 9 columns]

In [45]:
# Это будет json файл рядом с кодом
indefecators = {
    "Capacity": {"uf","nf","pf","mf"},
    "Resistance": {},
    "Inductance": {},
}

In [52]:
Capacity_indefecator = set(["uf","nf","pf","mf"])
def is_this_column_Capacity(column):
    T = 0
    set_col = set(column)
    counter = 0
    for item in set_col:
        item = str(item)
        for indefecator in Capacity_indefecator:
            #print(item)
            if indefecator in item:
                counter += 1
    return counter > T

In [71]:
def is_this_column_(column, column_name):
    T = 0
    set_col = set(column)
    column_indefecator = indefecators[column_name]
    counter = 0
    for item in set_col:
        item = str(item)
        for indefecator in column_indefecator:
            #print(item)
            if indefecator in item:
                counter += 1
    return counter > T, counter

In [72]:
is_this_column_Capacity(testParser.data["Value"])

True

In [76]:
is_this_column_(testParser.data["Value"], "Resistance")

(True, 29)

In [77]:
is_this_column_(testParser.data["Value"], "Capacity")

(True, 24)

In [79]:
is_this_column_(testParser.data["Value"], "Inductance") # Кажется это не так

(True, 2)

In [54]:
is_this_column_Capacity(testParser.data["Tolerance"])

False

In [55]:
is_this_column_Capacity(testParser.data["Description"])

False

In [56]:
is_this_column_Capacity(testParser.data["Part Number"])

False

In [4]:
columns = ["Value", "Tolerance", "Part Number", "Description"]

testParser = Specification("data/Спецификации SNRM-7140.xlsx", columns_to_work_with=columns, rows_to_skip=2)

In [5]:
testParser.data

Value Tolerance Part Number                 Description
2    0.01uf/630v       0.2         NaN    SMD Capacitor 1206(3216)
3     0.1uf/100v       0.2         NaN    SMD Capacitor 1206(3216)
4      0.1uf/16v       0.1         NaN    SMD Capacitor 0603(1608)
5      0.1uf/16v       0.2         NaN    SMD Capacitor 0603(1608)
6      0.1uf/16v       0.2         NaN    SMD Capacitor 0805(2012)
..           ...       ...         ...                         ...
134          NaN       NaN   1PS76SB10                       Diode
135          NaN       NaN  BZX84-C4V3                 Zener Diode
136          NaN       NaN      LM224D  Quad Operational Amplifier
137          NaN       NaN   PAM8406DR   5W STEREO AUDIO AMPLIFIER
138          NaN       NaN  XH-D-03-LF     Discrete Wire Connector

[137 rows x 4 columns]

In [6]:
testParser.make_conformity("Description","Component")
testParser.make_conformity("Tolerance","Tolerance")

In [7]:
testParser.conformity

{'Component': ['Description'],
 'Capacity': ['Value', 'Tolerance', 'Part Number', 'Description'],
 'Resistance': ['Value', 'Tolerance', 'Part Number', 'Description'],
 'Inductance': ['Value', 'Tolerance', 'Part Number', 'Description'],
 'Tolerance': ['Tolerance'],
 'Body': ['Value', 'Tolerance', 'Part Number', 'Description'],
 'Value': ['Value', 'Tolerance', 'Part Number', 'Description'],
 'Location': ['Value', 'Tolerance', 'Part Number', 'Description']}

In [8]:
testParser.parse()

In [9]:
testParser.parsed_data

Component Capacity Resistance Inductance Tolerance  \
0      SMD Capacitor 1206(3216)   [None]     [None]     [None]       0.2   
1      SMD Capacitor 1206(3216)   [None]     [None]     [None]       0.2   
2      SMD Capacitor 0603(1608)   [None]     [None]     [None]       0.1   
3      SMD Capacitor 0603(1608)   [None]     [None]     [None]       0.2   
4      SMD Capacitor 0805(2012)   [None]     [None]     [None]       0.2   
..                          ...      ...        ...        ...       ...   
132                       Diode   [None]     [None]     [None]       NaN   
133                 Zener Diode   [None]     [None]     [None]       NaN   
134  Quad Operational Amplifier   [None]     [None]     [None]       NaN   
135   5W STEREO AUDIO AMPLIFIER   [None]     [None]     [None]       NaN   
136     Discrete Wire Connector   [None]     [None]     [None]       NaN   

            Body        Value     Location  
0    0.01uf/630v  0.01uf/630v  0.01uf/630v  
1     0.1uf/100v   0.1uf/100v   0.1uf/100v  
2      0.1uf/16v    0.1uf/16v    0.1uf/16v  
3      0.1uf/16v    0.1uf/16v    0.1uf/16v  
4      0.1uf/16v    0.1uf/16v    0.1uf/16v  
..           ...          ...          ...  
132          NaN          NaN          NaN  
133          NaN          NaN          NaN  
134          NaN          NaN          NaN  
135          NaN          NaN          NaN  
136          NaN          NaN          NaN  

[137 rows x 8 columns]

In [10]:
import numpy as np
details = []
details_name = set()
estimate = collections.Counter()
for i in range(len(D)):
    current_row = D.iloc[i]
    name = current_row.Description
    if current_row["Part Number"] is not np.nan:
        name += f' - {current_row["Part Number"]}'
    estimate[name] += current_row.Number
    if name not in details_name:
        details_name.add(name)
        details.append(
            {
                "Description": current_row["Description"],
                "Part Number": current_row["Part Number"],
                "Value":  current_row["Value"],
                "Tolerance": current_row["Tolerance"],
            }
        )

NameError: name 'D' is not defined

In [ ]:
testParser.data

In [ ]:
a = [1,1,1]

In [ ]:
**a

In [ ]:
a = np.array([[1,1],1])
a.reshape(-1,1)

In [61]:
todo = pd.DataFrame(
columns=[
    "Component_type",
    "Name",
    "Capacity",
    "Resistance",
    "Inductance",
    "Tolerance",
    "Body",
    "Value",
    "Location",
    ])
todo = todo.append(
    {
    "Component_type": "Дискретное, на основе других столбцов",
    "Name": "-",
    "Capacity": "+",
    "Resistance": "+",
    "Inductance": "+",
    "Tolerance": "-",
    "Body": "-",
    "Value": "-",
    "Location": "Not in MVP",
    },ignore_index=True)
todo

Component_type Name Capacity Resistance Inductance  \
0  Дискретное, на основе других столбцов    -        +          +          +   

  Tolerance Body Value    Location  
0         -    -     -  Not in MVP

In [82]:
todo.to_excel("components.xlsx")